In [30]:
import pandas as pd
from haversine import haversine
import requests

In [31]:
data_path = "./data/"

In [34]:
def get_dong(latitude, longitude):
    dong = pd.read_csv(data_path + "dong.csv")
    
    url = "https://apis.openapi.sk.com/tmap/geo/reversegeocoding?version=1"

    payload = {
        "appKey": "l7xx0ffb87c22bdb45ae8facaeeb7958ad36",
        "lon": longitude,
        "lat": latitude,
    }

    res = requests.get(url, params=payload)

    try:
        dongCode = int(json.loads(res.text)["addressInfo"]["legalDongCode"][5:8])
    except:
        return "ERROR"

    if dongCode in dong["dongCode"].unique():
        return dongCode
    else:
        return "Invalid Code"

In [35]:
# mariaDB 쿼리로 처리할 부분
# dongCode가 같은 것만 불러오기
def return_df_dongCode(dongCode):
    cautionzone = pd.read_csv(data_path + "cautionzone.csv")
    parkingzone = pd.read_csv(data_path + "parkingzone.csv")
    
    cautionzone = cautionzone[cautionzone["dongCode"]==dongCode].reset_index(drop=True)
    parkingzone = parkingzone[parkingzone["dongCode"]==dongCode].reset_index(drop=True)
    
    return cautionzone, parkingzone

In [38]:
# 기준 거리
criteria = 100

# 현 위치
latitude = 37.50967163551159
longitude = 127.05554547579632
here = (latitude, longitude)

# 해당 구역 저장
place_list = []


dongCode = str(get_dong(latitude, longitude))
cautionzone, parkingzone = return_df_dongCode(dongCode) ###

### 데이터프레임 --> DB로 변경 필요
for i in range(len(cautionzone)):
    zone = cautionzone.loc[i, :]
    distance = haversine(here, (zone["latitude"], zone["longitude"]), unit="m")
    if distance < criteria:
        print(zone["type"], zone["name"])
        place_list.append({"type": zone["type"], "latitude": zone["latitude"], "longitude": zone["longitude"]})

for i in range(len(parkingzone)):
    zone = cautionzone.loc[i, :]
    distance = haversine(here, (zone["latitude"], zone["longitude"]), unit="m")
    if distance < criteria:
        print(zone["type"], zone["name"])
        place_list.append({"type": zone["type"], "latitude": zone["latitude"], "longitude": zone["longitude"]})        

bus 한별구립어린이집
fire 휴비스 사옥
fire nan
